In [2]:
!rm -rf DeSTA2

In [3]:
# Setup completo para Google Colab
!pip install -q transformers accelerate bitsandbytes
!git clone https://github.com/kehanlu/DeSTA2.git
%cd DeSTA2

Cloning into 'DeSTA2'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 86 (delta 23), reused 72 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (86/86), 4.54 MiB | 11.58 MiB/s, done.
Resolving deltas: 100% (23/23), done.
/content/DeSTA2/DeSTA2


In [4]:
# Parche para el bug
import transformers.models.bert.modeling_bert as bert_modeling

original_init = bert_modeling.BertAttention.__init__

def patched_init(self, config, *args, **kwargs):
    if hasattr(config, '_attn_implementation'):
        if config._attn_implementation is None or config._attn_implementation == "torch":
            config._attn_implementation = 'eager'
    return original_init(self, config, *args, **kwargs)

bert_modeling.BertAttention.__init__ = patched_init

In [5]:
# Autenticación
from huggingface_hub import login
login(token="hf_xxxx")  # ← CAMBIAR ESTO

In [6]:
# Cargar modelo
from desta import DestaModel
import torch

model = DestaModel.from_pretrained(
    "DeSTA-ntu/DeSTA2-8B-beta",
    load_in_8bit=True,
    device_map="auto"
)

print(" Listo para usar!")

`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

qformer_connector.pth:   0%|          | 0.00/89.0M [00:00<?, ?B/s]

 Listo para usar!


In [7]:
# Los archivos de ejemplo están en /content/DeSTA2/assets/audios/
audio_path = "/content/DeSTA2/assets/audios/7_1_d7.wav"

messages = [
    {"role": "system", "content": "Focus on the audio clip."},
    {"role": "audio", "content": audio_path},
    {"role": "user", "content": "Describe the audio clip."},
]

print(" Generando respuesta...")
generated_ids = model.chat(messages, max_new_tokens=256, do_sample=True, temperature=0.6, top_p=0.9)
response = model.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("\n Respuesta:")
print(response)

 Generando respuesta...


The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will 


 Respuesta:
The audio clip is a brief recording of a female speaker saying "Thank you for calling. Goodbye." in a happy tone. The clip lasts for 2 seconds.


In [ ]:
!pip install datasets pyarrow==16.0.0 --quiet

import datasets

In [ ]:
dataset_name = "DynamicSuperb/NoiseSNRLevelPrediction_VCTK_MUSAN-Music"
data = datasets.load_dataset(dataset_name)

print(f" Dataset '{dataset_name}' loaded successfully!")
print(data)

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

 Dataset 'DynamicSuperb/SpeechSentimentAnalysis_MELD' loaded successfully!
DatasetDict({
    test: Dataset({
        features: ['audio', 'file', 'instruction', 'label', 'transcription'],
        num_rows: 200
    })
})


In [10]:
!pip install soundfile

In [11]:
from IPython.display import Audio
import numpy as np

# Cargar el dataset
dataset_name = "DynamicSuperb/NoiseSNRLevelPrediction_VCTK_MUSAN-Music"
data = datasets.load_dataset(dataset_name)

# Ver la estructura primero
print("Splits disponibles:", list(data.keys()))
print("\nColumnas:", data[list(data.keys())[0]].column_names)

# Buscar el archivo específico
split_name = list(data.keys())[0]  # Usar el primer split
dataset_split = data[split_name]

# Buscar por nombre de archivo
target_file = "clean_VCTK-Corpus_16k_p239_502.wav"

for idx, example in enumerate(dataset_split):
    # Verificar si el nombre del archivo está en alguna columna
    if any(target_file in str(value) for value in example.values()):
        print(f" Encontrado en índice {idx}")
        print(f"Ejemplo completo: {example}")
        
        # El audio suele estar en una columna llamada 'audio' o 'file'
        if 'audio' in example:
            audio_data = example['audio']
            
            # Guardar el archivo
            import soundfile as sf
            sf.write(f"{target_file}", audio_data['array'], audio_data['sampling_rate'])
            print(f" Archivo guardado como {target_file}")
            
            # Reproducir en el notebook
            display(Audio(audio_data['array'], rate=audio_data['sampling_rate']))
        break

Splits disponibles: ['test']

Columnas: ['file', 'audio', 'instruction', 'label']
 Encontrado en índice 0
Ejemplo completo: {'file': 'clean_VCTK-Corpus_16k_p239_502.wav', 'audio': {'path': None, 'array': array([-0.00869751, -0.01580811, -0.01412964, ...,  0.14901733,
        0.22567749,  0.31292725]), 'sampling_rate': 16000}, 'instruction': 'Identify the SNR of the utterance affected by music noise. The answer could be zero, five, ten, fifteen, or clean.', 'label': 'clean'}
 Archivo guardado como clean_VCTK-Corpus_16k_p239_502.wav


In [13]:
import os

# Después de guardar el archivo con sf.write
archivo_guardado = f"{target_file}"
ruta_completa = os.path.abspath(archivo_guardado)

print(f" Ruta completa: {ruta_completa}")
print(f" Directorio actual: {os.getcwd()}")

# Para listar archivos en el directorio actual
print(f"\n Archivos .wav en el directorio:")
!ls -lh *.wav

 Ruta completa: /content/DeSTA2/DeSTA2/clean_VCTK-Corpus_16k_p239_502.wav
 Directorio actual: /content/DeSTA2/DeSTA2

 Archivos .wav en el directorio:
-rw-r--r-- 1 root root 126K Dec 17 03:44 clean_VCTK-Corpus_16k_p239_502.wav


In [ ]:
audio_path = "/content/DeSTA2/DeSTA2/clean_VCTK-Corpus_16k_p239_502.wav"

messages = [
    {"role": "system", "content": "Focus on the audio clip."},
    {"role": "audio", "content": audio_path},
    {"role": "user", "content": "Describe the audio clip."},
]

print(" Generando respuesta...")
generated_ids = model.chat(messages, max_new_tokens=256, do_sample=True, temperature=0.6, top_p=0.9)
response = model.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("\n Respuesta:")
print(response)

 Generando respuesta...

 Respuesta:
The audio clip is a 4-second long audio recording of a female speaker saying "McPherson came from United Biscuits." The speaker's tone is neutral, indicating a matter-of-fact delivery. There is no apparent emotion or inflection in the speaker's voice. The audio quality is clear and crisp, with no background noise or distortion.
